In [2]:
!pip install qiskit qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00


In [3]:
# Task 1: Test with different secret strings
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    return most

# Test with different secret strings
test_strings = ['1011', '1111', '0000', '1010']

for s in test_strings:
    print(f'\nSecret string s = {s}')
    qc = bernstein_vazirani_circuit(s)
    measured = run_bv(qc)
    measured_reversed = measured[::-1]
    print(f'Measured (reversed): {measured_reversed}')

    if measured_reversed == s:
        print('✅ Match!')
    else:
        print('❌ Mismatch')


Secret string s = 1011
Counts: {'1101': 1024}
Measured (reversed): 1011
✅ Match!

Secret string s = 1111
Counts: {'1111': 1024}
Measured (reversed): 1111
✅ Match!

Secret string s = 0000
Counts: {'0000': 1024}
Measured (reversed): 0000
✅ Match!

Secret string s = 1010
Counts: {'0101': 1024}
Measured (reversed): 1010
✅ Match!


In [4]:
# Task 2: Implement f(x) = s·x ⊕ b with constant bit b
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle_with_b(qc, inputs, ancilla, s, b):
    """Implements oracle for f(x) = s · x ⊕ b."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

    if b == '1':
        qc.x(ancilla)

def bernstein_vazirani_circuit_with_b(s, b):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle_with_b(qc, inputs, ancilla, s, b)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv_with_b(qc, shots=1024):
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts

# Test with b = 0 and b = 1
s = '1011'

for b in ['0', '1']:
    print(f'\nTesting: s = {s}, b = {b}')
    qc = bernstein_vazirani_circuit_with_b(s, b)
    print(qc.draw())

    counts = run_bv_with_b(qc)
    print('Counts:', counts)
    plot_histogram(counts, title=f's={s}, b={b}')
    plt.show()

    most = max(counts, key=counts.get)
    measured_reversed = most[::-1]
    print(f'Measured (reversed): {measured_reversed}')
    print(f'Note: b={b} does not affect the measured result for s')


Testing: s = 1011, b = 0
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Measured (reversed): 1011
Note: b=0 does not affect the measured result for s

Testing: s = 1011, b = 1
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫──────────

In [5]:
# Task 3: Add noise using NoiseModel and analyze robustness
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def bv_oracle(qc, inputs, ancilla, s):
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle(qc, inputs, ancilla, s)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def create_noise_model(error_rate):
    noise_model = NoiseModel()
    error_1q = depolarizing_error(error_rate, 1)
    noise_model.add_all_qubit_quantum_error(error_1q, ['h', 'x'])
    error_2q = depolarizing_error(error_rate * 2, 2)
    noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])
    return noise_model

def run_bv_with_noise(qc, noise_model=None, shots=1024):
    sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return counts

# Test with different noise levels
s = '1011'
error_rates = [0.0, 0.01, 0.05, 0.1]
shots = 2048

print(f'Secret string s = {s}\n')

for error_rate in error_rates:
    print(f'Error Rate: {error_rate:.2%}')
    qc = bernstein_vazirani_circuit(s)

    noise_model = None if error_rate == 0.0 else create_noise_model(error_rate)
    counts = run_bv_with_noise(qc, noise_model, shots)

    most = max(counts, key=counts.get)
    measured_reversed = most[::-1]
    success_rate = counts[most] / shots * 100

    print(f'Measured: {measured_reversed}, Success Rate: {success_rate:.1f}%')
    print(f'Top results: {dict(sorted(counts.items(), key=lambda x: x[1], reverse=True)[:3])}')

    plot_histogram(counts, title=f'Error Rate: {error_rate:.2%}')
    plt.show()
    print()

Secret string s = 1011

Error Rate: 0.00%
Measured: 1011, Success Rate: 100.0%
Top results: {'1101': 2048}

Error Rate: 1.00%
Measured: 1011, Success Rate: 91.2%
Top results: {'1101': 1868, '0101': 61, '1001': 33}

Error Rate: 5.00%
Measured: 1011, Success Rate: 65.6%
Top results: {'1101': 1343, '0101': 200, '1100': 132}

Error Rate: 10.00%
Measured: 1011, Success Rate: 47.8%
Top results: {'1101': 978, '0101': 248, '1001': 186}



In [6]:
# Task 4: Educational notebook with step-by-step explanations
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

print("="*60)
print("BERNSTEIN-VAZIRANI ALGORITHM - STEP BY STEP")
print("="*60)

s = '101'
n = len(s)

print(f"\nSecret string: s = {s}")
print(f"Goal: Recover s using only 1 quantum query\n")

# Step 1: Initialize
print("STEP 1: Initialize qubits")
qreg = QuantumRegister(n + 1, 'q')
creg = ClassicalRegister(n, 'c')
qc = QuantumCircuit(qreg, creg)
qc.x(n)
print("- Input qubits initialized to |0⟩")
print("- Ancilla qubit initialized to |1⟩")
print(qc.draw())

# Step 2: Hadamard gates
print("\nSTEP 2: Apply Hadamard gates to create superposition")
qc.h(qreg)
qc.barrier()
print("- All qubits now in superposition")
print("- Input qubits: equal superposition of all n-bit strings")
print("- Ancilla: in state |-⟩ = (|0⟩-|1⟩)/√2")
print(qc.draw())

# Step 3: Oracle
print("\nSTEP 3: Apply oracle")
print(f"Oracle implements f(x) = s·x where s = {s}")
for i, bit in enumerate(s):
    if bit == '1':
        qc.cx(i, n)
        print(f"- Apply CNOT: q_{i} → ancilla (because s[{i}]=1)")
qc.barrier()
print("- Oracle creates phase kickback based on s·x")
print(qc.draw())

# Step 4: Hadamard again
print("\nSTEP 4: Apply Hadamard to input qubits")
for q in range(n):
    qc.h(q)
print("- Second Hadamard causes interference")
print("- Correct answer (s) amplified, others cancel")
print(qc.draw())

# Step 5: Measure
print("\nSTEP 5: Measure input qubits")
qc.measure(range(n), creg)
print(qc.draw())

# Run circuit
print("\n" + "="*60)
print("EXECUTION")
print("="*60)
sim = AerSimulator()
tqc = transpile(qc, sim)
job = sim.run(tqc, shots=1024)
result = job.result()
counts = result.get_counts()

print(f"Counts: {counts}")
plot_histogram(counts, title=f'Results for s={s}')
plt.show()

most = max(counts, key=counts.get)
measured_reversed = most[::-1]

print(f"\nSecret string s: {s}")
print(f"Measured result:  {measured_reversed}")

if measured_reversed == s:
    print("✅ Successfully recovered secret string in 1 query!")
else:
    print("Note: Check bit ordering")

print("\n" + "="*60)
print("KEY POINTS")
print("="*60)
print("1. Classical algorithm needs n queries")
print("2. Quantum algorithm needs only 1 query")
print("3. Uses superposition and interference")
print("4. Result is deterministic (100% success)")

BERNSTEIN-VAZIRANI ALGORITHM - STEP BY STEP

Secret string: s = 101
Goal: Recover s using only 1 quantum query

STEP 1: Initialize qubits
- Input qubits initialized to |0⟩
- Ancilla qubit initialized to |1⟩
          
q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘
c: 3/═════
          

STEP 2: Apply Hadamard gates to create superposition
- All qubits now in superposition
- Input qubits: equal superposition of all n-bit strings
- Ancilla: in state |-⟩ = (|0⟩-|1⟩)/√2
     ┌───┐      ░ 
q_0: ┤ H ├──────░─
     ├───┤      ░ 
q_1: ┤ H ├──────░─
     ├───┤      ░ 
q_2: ┤ H ├──────░─
     ├───┤┌───┐ ░ 
q_3: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 3/═════════════
                  

STEP 3: Apply oracle
Oracle implements f(x) = s·x where s = 101
- Apply CNOT: q_0 → ancilla (because s[0]=1)
- Apply CNOT: q_2 → ancilla (because s[2]=1)
- Oracle creates phase kickback based on s·x
     ┌───┐      ░            ░ 
q_0: ┤ H ├──────░───■────────░─
     ├───┤     